In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
from datetime import datetime

pd.set_option('display.max_row', None)

In [2]:
# required urls

url_ipad = 'https://biggeek.ru/catalog/planshety-apple-ipad'
url_iphone = 'https://biggeek.ru/catalog/apple-iphone'
url_mac = 'https://biggeek.ru/catalog/noutbuki-apple'
url_watch = 'https://biggeek.ru/catalog/apple-watch'

In [3]:
# parcing ipad data

response = requests.get(url_ipad)
bs = BeautifulSoup(response.text, 'lxml')
page_number = int(bs.findAll('a', class_='prod-pagination__item')[-1].text)

data_ipad = []

for page in range(1, page_number+1):

  url = f'{url_ipad}?page={page}'
  response = requests.get(url)
  bs = BeautifulSoup(response.text, 'lxml')

  sleep(3)
  
  prod = bs.findAll('div', class_='catalog-card')

  for p in prod:
    name = p.find('a', class_='catalog-card__title cart-modal-title').text
    price = p.find('b', class_='cart-modal-count').text
    old_price = p.find('span', class_='old-price').text

    data_ipad.append([name, price, old_price])

In [4]:
# parcing iphone data

response = requests.get(url_iphone)
bs = BeautifulSoup(response.text, 'lxml')
page_number = int(bs.findAll('a', class_='prod-pagination__item')[-1].text)

data_iphone = []

for page in range(1, page_number+1):

  url = f'{url_iphone}?page={page}'
  response = requests.get(url)
  bs = BeautifulSoup(response.text, 'lxml')

  sleep(3)
  
  prod = bs.findAll('div', class_='catalog-card')

  for p in prod:
    name = p.find('a', class_='catalog-card__title cart-modal-title').text
    price = p.find('b', class_='cart-modal-count').text
    old_price = p.find('span', class_='old-price').text

    data_iphone.append([name, price, old_price])

In [5]:
# parcing mac data

response = requests.get(url_mac)
bs = BeautifulSoup(response.text, 'lxml')
page_number = int(bs.findAll('a', class_='prod-pagination__item')[-1].text)

data_mac = []

for page in range(1, page_number+1):

  url = f'{url_mac}?page={page}'
  response = requests.get(url)
  bs = BeautifulSoup(response.text, 'lxml')

  sleep(3)
  
  prod = bs.findAll('div', class_='catalog-card')

  for p in prod:
    name = p.find('a', class_='catalog-card__title cart-modal-title').text
    price = p.find('b', class_='cart-modal-count').text
    old_price = p.find('span', class_='old-price').text

    data_mac.append([name, price, old_price])

In [6]:
# parcing watch data

response = requests.get(url_watch)
bs = BeautifulSoup(response.text, 'lxml')
page_number = int(bs.findAll('a', class_='prod-pagination__item')[-1].text)

data_watch = []

for page in range(1, page_number+1):

  url = f'{url_watch}?page={page}'
  response = requests.get(url)
  bs = BeautifulSoup(response.text, 'lxml')

  sleep(3)
  
  prod = bs.findAll('div', class_='catalog-card')

  for p in prod:
    name = p.find('a', class_='catalog-card__title cart-modal-title').text
    price = p.find('b', class_='cart-modal-count').text
    old_price = p.find('span', class_='old-price').text

    data_watch.append([name, price, old_price])

In [7]:
# creation DataFrame

all_data = data_ipad + data_iphone + data_mac + data_watch

df = pd.DataFrame(all_data, columns=['name', 'current_price', 'old_price'])

df = df[~(df['name'].str.contains('Клавиатура')) & ~(df['name'].str.contains('Стилус'))].reset_index(drop=True)

In [8]:
# adding new columns

df['current_price'] = df['current_price'].str.strip(' ₽').str.replace(' ', '').str.replace('от', '').astype(int)

df['old_price'] = df['old_price'].str.strip().str.replace(' ', '').replace('', float('NaN')).astype(float)
df['old_price'] = df['old_price'].fillna(df['current_price'])

df['discount'] = df['old_price'] - df['current_price']
df['is_discount'] = df['discount'].map(lambda x: 'Да' if x > 0 else 'Нет')

df['type'] = df['name'].map(lambda x: x.split()[1])
# df['color'] = df['name'].map(lambda x: x.split()[-1] if x.split()[1] != 'Watch' else '-')
df['is_cellular'] = df['name'].map(lambda x: 'Да' if 'cellular' in x.lower() else 'Нет')

df['memory_gb'] = ( df['name']
    .map(lambda x: x[x.lower().find('gb')-3:x.lower().find('gb')] if 'gb' in x.lower() else (x[x.lower().find('tb')-3:x.lower().find('tb')] if 'tb' in x.lower() else '0'))
    .str.strip(')')
    .str.strip('x')
    .str.strip('o')
    .str.strip(',')
    .str.strip()
    .astype(int)
)
df['memory_gb'] = df['memory_gb'].map(lambda x: x * 1024 if x in [1, 2] else x)

In [9]:
def get_color(row):

    'The function gets the color from the "name" column'

    row['name'] = row['name'].upper()
    
    if 'SPACE GRAY' in row['name'] or 'СЕРЫЙ' in row['name'] or 'PACE GREY' in row['name']:
        return 'Space Gray'
    elif 'PINK' in row['name']:
        return 'Pink'
    elif 'YELLOW' in row['name'] or 'ЖЁЛТЫЙ' in row['name']  or 'ЖЁЛТОГО' in row['name']:
        return 'Yellow'
    elif 'STARLIGHT' in row['name'] or 'ЗВЕЗДА' in row['name']:
        return 'Starlight'
    elif 'SILVER' in row['name'] or 'СЕРЕБРИСТОГО' in row['name']:
        return 'Silver'
    elif 'SKY BLUE' in row['name']:
        return 'Sky Blue'
    elif 'SIERRA BLUE' in row['name']:
        return 'Sierra Blue'
    elif 'SPACE BLUE' in row['name']:
        return 'Space Blue'
    elif 'PACIFIC BLUE' in row['name']:
        return 'Pacific Blue'
    elif 'BLUE' in row['name'] or 'СИНЕГО' in row['name']:
        return 'Blue'
    elif 'DEEP PURPLE' in row['name']:
        return 'Deep Purple'
    elif 'PURPLE' in row['name']:
        return 'Purple'
    elif 'ROSE GOLD' in row['name']:
        return 'Rose Gold'
    elif 'GOLD' in row['name'] or 'ЗОЛОТОГО' in row['name']:
        return 'Gold'
    elif 'ROSE GOLD' in row['name']:
        return 'Rose Gold'
    elif 'SPACE BLACK' in row['name']:
        return 'Space Black'
    elif 'ALPINE GREEN' in row['name']:
        return 'Alpine Green'
    elif 'GREEN' in row['name'] or 'ЗЕЛЁНОГО' in row['name'] or 'ЗЕЛЁНЫЙ' in row['name']:
        return 'Green'
    elif 'MIDNIGHT' in row['name']:
        return 'Midnight'
    elif 'RED' in row['name']:
        return 'RED'
    elif 'BLACK' in row['name'] or 'НОЧЬ' in row['name']:
        return 'Black'
    elif 'GRAPHITE' in row['name'] or 'ГРАФИТ' in row['name'] or '«ГРАФИТ»' in row['name']:
        return 'Graphite'
    elif 'WHITE' in row['name'] or 'БЕЛОГО' in row['name']:
        return 'White'
    elif 'CORAL' in row['name']:
        return 'Coral'
    elif 'ОРАНЖЕВОГО' in row['name']:
        return 'Orange'
    else:
        return '-'

df['color'] = df.apply(get_color, axis=1)

In [10]:
# adding the update date column

df['update_date'] = datetime.now().strftime('%Y-%m-%d %H:%M')

In [11]:
# changing the data type

for col in ['old_price', 'discount']:
    df[col] = df[col].astype(int)

df['update_date'] = pd.to_datetime(df['update_date'])

In [12]:
df[:10]

,name,current_price,old_price,discount,is_discount,type,is_cellular,memory_gb,color,update_date
0,"Apple iPad Pro 12.9"" (2022) 128GB Wi-Fi Space ...",87990,99990,12000,Да,iPad,Нет,128,Space Gray,2023-02-18 20:57:00
1,"Apple iPad Pro 11"" (2022) 128GB Wi-Fi Space Gray",71990,79990,8000,Да,iPad,Нет,128,Space Gray,2023-02-18 20:57:00
2,"Apple iPad 10.9"" (2022) 64GB Wi-Fi Yellow",43990,43990,0,Нет,iPad,Нет,64,Yellow,2023-02-18 20:57:00
3,"Apple iPad Pro 11"" (2021) 128Gb Wi-Fi Space Gray",63990,68990,5000,Да,iPad,Нет,128,Space Gray,2023-02-18 20:57:00
4,"Apple iPad Pro 12.9"" (2022) 128GB Wi-Fi Silver",87990,102990,15000,Да,iPad,Нет,128,Silver,2023-02-18 20:57:00
5,"Apple iPad Pro 11"" (2022) 128GB Wi-Fi Silver",71990,79990,8000,Да,iPad,Нет,128,Silver,2023-02-18 20:57:00
6,"Apple iPad 10.9"" (2022) 64GB Wi-Fi Pink",40990,40990,0,Нет,iPad,Нет,64,Pink,2023-02-18 20:57:00
7,Apple iPad Air (2022) 64Gb Wi-Fi + Cellular St...,59490,81990,22500,Да,iPad,Да,64,Starlight,2023-02-18 20:57:00
8,"Apple iPad 10.2"" (2021) 64GB Wi-Fi Space Gray",29490,31990,2500,Да,iPad,Нет,64,Space Gray,2023-02-18 20:57:00
9,Apple iPad mini (2021) 64GB Wi-Fi Space Gray,44990,44990,0,Нет,iPad,Нет,64,Space Gray,2023-02-18 20:57:00
